# Quantum Chemistry Hamiltonians

In [1]:
!pip install block2==0.5.1 -qq --progress-bar off
!pip install pyscf==2.1.1 -qq --progress-bar off

## Introduction

In this tutorial we explain how to perform quantum chemistry DMRG using the python interface of ``block2``.

The quantum chemistry Hamiltonian in its second quantized form has to be defined in a set of orbitals, such as the Hartree-Fock orbitals. The symmetry that can be used in the DMRG calculation is thus has a dependence on the symmetry of the Hartree-Fock orbitals.

1. For spin-restricted Hartree-Fock (RHF) orbitals, we can perform spin-adapted DMRG (``SU2`` mode in ``block2``) or non-spin-adapted DMRG with any lower symmetries (``SZ`` or ``SGF``).

2. For spin-unrestricted Hartree-Fock (UHF) orbitals, we can perform non-spin-adapted DMRG (``SZ`` mode in ``block2``) or DMRG with lower symmetries (such as ``SGF``).

3. For general Hartree-Fock (GHF) orbitals, we can perform DMRG in spin-orbitals (``SGF`` mode in ``block2``) or first translate the Hamiltonian into the qubit Hamiltonian then do DMRG (``SGB`` mode in ``block2``).

4. For relativistic Dirac Hartree-Fock (DHF) orbitals, we can perform DMRG in complex spin-orbitals (``SGFCPX`` mode in ``block2``).

Next, we will explain how to set up the integrals and perform DMRG in each of the modes (1) (2) (3) and (4). The quantum chemistry integrals will be generated using ``pyscf`` and transformed using funtions defined in ``pyblock2._pyscf.ao2mo``.


In [2]:
import numpy as np
from pyblock2._pyscf.ao2mo import integrals as itg
from pyblock2.driver.core import DMRGDriver, SymmetryTypes

## Spin-Restricted Integrals

Here we use ``get_rhf_integrals`` function to get the integrals. Note that in order to do DMRG in a CASCI space, one can set the ``ncore`` (number of core orbitals) and ``ncas`` (number of active orbitals) parameters in ``get_*_integrals``. ``ncas=None`` will include all orbitals in DMRG.

For medium to large scale DMRG calculations, it is highly recommended to use a scratch space with high IO speed rather than the ``./tmp`` used in the following example. One also need to set a suitable ``stack_mem`` in the ``DMRGDriver`` constructor to set the memory used for storing renormalized operators (in bytes). The default is ``stack_mem=int(1024**3)`` (1 GB). For medium scale calculations 10 to 30 GB might be required.

In [3]:
from pyscf import gto, scf

mol = gto.M(atom="N 0 0 0; N 0 0 1.1", basis="sto3g", symmetry="d2h", verbose=0)
mf = scf.RHF(mol).run(conv_tol=1E-14)
ncas, n_elec, spin, ecore, h1e, g2e, orb_sym = itg.get_rhf_integrals(mf,
    ncore=0, ncas=None, g2e_symm=8)

driver = DMRGDriver(scratch="./tmp", symm_type=SymmetryTypes.SU2, n_threads=4)
driver.initialize_system(n_sites=ncas, n_elec=n_elec, spin=spin, orb_sym=orb_sym)

mpo = driver.get_qc_mpo(h1e=h1e, g2e=g2e, ecore=ecore, iprint=1)

def run_dmrg(driver, mpo):
    ket = driver.get_random_mps(tag="GS", bond_dim=250, nroots=1)
    bond_dims = [250] * 4 + [500] * 4
    noises = [1e-4] * 4 + [1e-5] * 4 + [0]
    thrds = [1e-10] * 8
    return driver.dmrg(
        mpo,
        ket,
        n_sweeps=20,
        bond_dims=bond_dims,
        noises=noises,
        thrds=thrds,
        iprint=1,
    )

energies = run_dmrg(driver, mpo)
print('DMRG energy = %20.15f' % energies)

integral symmetrize error =  6.068111737226223e-14
integral cutoff error =  0.0
mpo terms =        972

Build MPO | Nsites =    10 | Nterms =        972 | Algorithm = FastBIP | Cutoff = 1.00e-20
 Site =     0 /    10 .. Mmpo =    13 DW = 0.00e+00 NNZ =       13 SPT = 0.0000 Tmvc = 0.000 T = 0.011
 Site =     1 /    10 .. Mmpo =    34 DW = 0.00e+00 NNZ =       59 SPT = 0.8665 Tmvc = 0.000 T = 0.004
 Site =     2 /    10 .. Mmpo =    56 DW = 0.00e+00 NNZ =      117 SPT = 0.9386 Tmvc = 0.001 T = 0.008
 Site =     3 /    10 .. Mmpo =    70 DW = 0.00e+00 NNZ =      363 SPT = 0.9074 Tmvc = 0.000 T = 0.007
 Site =     4 /    10 .. Mmpo =    80 DW = 0.00e+00 NNZ =      217 SPT = 0.9613 Tmvc = 0.000 T = 0.006
 Site =     5 /    10 .. Mmpo =    94 DW = 0.00e+00 NNZ =      201 SPT = 0.9733 Tmvc = 0.000 T = 0.004
 Site =     6 /    10 .. Mmpo =    54 DW = 0.00e+00 NNZ =      169 SPT = 0.9667 Tmvc = 0.000 T = 0.003
 Site =     7 /    10 .. Mmpo =    30 DW = 0.00e+00 NNZ =       73 SPT = 0.9549 Tmvc

We can also run non-spin-adapted DMRG (``SZ`` mode) using the restricted integrals.

In [4]:
driver = DMRGDriver(scratch="./tmp", symm_type=SymmetryTypes.SZ, n_threads=4)
driver.initialize_system(n_sites=ncas, n_elec=n_elec, spin=spin, orb_sym=orb_sym)

mpo = driver.get_qc_mpo(h1e=h1e, g2e=g2e, ecore=ecore, iprint=1)
energies = run_dmrg(driver, mpo)
print('DMRG energy = %20.15f' % energies)

integral symmetrize error =  7.089394717975372e-14
integral cutoff error =  0.0
mpo terms =       2778

Build MPO | Nsites =    10 | Nterms =       2778 | Algorithm = FastBIP | Cutoff = 1.00e-20
 Site =     0 /    10 .. Mmpo =    26 DW = 0.00e+00 NNZ =       26 SPT = 0.0000 Tmvc = 0.000 T = 0.008
 Site =     1 /    10 .. Mmpo =    66 DW = 0.00e+00 NNZ =      143 SPT = 0.9167 Tmvc = 0.001 T = 0.005
 Site =     2 /    10 .. Mmpo =   110 DW = 0.00e+00 NNZ =      283 SPT = 0.9610 Tmvc = 0.001 T = 0.006
 Site =     3 /    10 .. Mmpo =   138 DW = 0.00e+00 NNZ =     1023 SPT = 0.9326 Tmvc = 0.001 T = 0.011
 Site =     4 /    10 .. Mmpo =   158 DW = 0.00e+00 NNZ =      535 SPT = 0.9755 Tmvc = 0.001 T = 0.016
 Site =     5 /    10 .. Mmpo =   186 DW = 0.00e+00 NNZ =      463 SPT = 0.9842 Tmvc = 0.000 T = 0.011
 Site =     6 /    10 .. Mmpo =   106 DW = 0.00e+00 NNZ =      415 SPT = 0.9790 Tmvc = 0.000 T = 0.011
 Site =     7 /    10 .. Mmpo =    58 DW = 0.00e+00 NNZ =      163 SPT = 0.9735 Tmvc

We can also run DMRG in spin orbitals (``SGF`` mode) using the restricted integrals, which will be much slower.

In [5]:
driver = DMRGDriver(scratch="./tmp", symm_type=SymmetryTypes.SGF, n_threads=4)

driver.n_sites = ncas
g2e = driver.unpack_g2e(g2e)
orb_sym = [orb_sym[i // 2] for i in range(len(orb_sym) * 2)]
n_sites = ncas * 2

driver.initialize_system(n_sites=n_sites, n_elec=n_elec, spin=spin, orb_sym=orb_sym)

mpo = driver.get_qc_mpo(h1e=h1e, g2e=g2e, ecore=ecore, iprint=1)
energies = run_dmrg(driver, mpo)
print('DMRG energy = %20.15f' % energies)

integral symmetrize error =  7.089394717975372e-14
integral cutoff error =  0.0
mpo terms =       2438

Build MPO | Nsites =    20 | Nterms =       2438 | Algorithm = FastBIP | Cutoff = 1.00e-20
 Site =     0 /    20 .. Mmpo =     7 DW = 0.00e+00 NNZ =        7 SPT = 0.0000 Tmvc = 0.001 T = 0.013
 Site =     1 /    20 .. Mmpo =    20 DW = 0.00e+00 NNZ =       19 SPT = 0.8643 Tmvc = 0.000 T = 0.017
 Site =     2 /    20 .. Mmpo =    45 DW = 0.00e+00 NNZ =       45 SPT = 0.9500 Tmvc = 0.001 T = 0.012
 Site =     3 /    20 .. Mmpo =    62 DW = 0.00e+00 NNZ =      131 SPT = 0.9530 Tmvc = 0.001 T = 0.017
 Site =     4 /    20 .. Mmpo =    81 DW = 0.00e+00 NNZ =      159 SPT = 0.9683 Tmvc = 0.001 T = 0.010
 Site =     5 /    20 .. Mmpo =   104 DW = 0.00e+00 NNZ =      203 SPT = 0.9759 Tmvc = 0.001 T = 0.019
 Site =     6 /    20 .. Mmpo =   125 DW = 0.00e+00 NNZ =      265 SPT = 0.9796 Tmvc = 0.001 T = 0.014
 Site =     7 /    20 .. Mmpo =   126 DW = 0.00e+00 NNZ =      974 SPT = 0.9382 Tmvc

## The ``SZ`` Mode

Here we use ``get_uhf_integrals`` function to get the integrals.

In [6]:
from pyscf import gto, scf

mol = gto.M(atom="N 0 0 0; N 0 0 1.1", basis="sto3g", symmetry="d2h", verbose=0)
mf = scf.UHF(mol).run(conv_tol=1E-14)
ncas, n_elec, spin, ecore, h1e, g2e, orb_sym = itg.get_uhf_integrals(mf,
    ncore=0, ncas=None, g2e_symm=8)

driver = DMRGDriver(scratch="./tmp", symm_type=SymmetryTypes.SZ, n_threads=4)
driver.initialize_system(n_sites=ncas, n_elec=n_elec, spin=spin, orb_sym=orb_sym)

mpo = driver.get_qc_mpo(h1e=h1e, g2e=g2e, ecore=ecore, iprint=1)
energies = run_dmrg(driver, mpo)
print('DMRG energy = %20.15f' % energies)

integral symmetrize error =  1.6337544812742183e-13
integral cutoff error =  0.0
mpo terms =       2778

Build MPO | Nsites =    10 | Nterms =       2778 | Algorithm = FastBIP | Cutoff = 1.00e-20
 Site =     0 /    10 .. Mmpo =    26 DW = 0.00e+00 NNZ =       26 SPT = 0.0000 Tmvc = 0.000 T = 0.011
 Site =     1 /    10 .. Mmpo =    66 DW = 0.00e+00 NNZ =      143 SPT = 0.9167 Tmvc = 0.001 T = 0.008
 Site =     2 /    10 .. Mmpo =   110 DW = 0.00e+00 NNZ =      283 SPT = 0.9610 Tmvc = 0.001 T = 0.008
 Site =     3 /    10 .. Mmpo =   138 DW = 0.00e+00 NNZ =     1023 SPT = 0.9326 Tmvc = 0.001 T = 0.009
 Site =     4 /    10 .. Mmpo =   158 DW = 0.00e+00 NNZ =      535 SPT = 0.9755 Tmvc = 0.001 T = 0.007
 Site =     5 /    10 .. Mmpo =   186 DW = 0.00e+00 NNZ =      463 SPT = 0.9842 Tmvc = 0.000 T = 0.006
 Site =     6 /    10 .. Mmpo =   106 DW = 0.00e+00 NNZ =      415 SPT = 0.9790 Tmvc = 0.000 T = 0.006
 Site =     7 /    10 .. Mmpo =    58 DW = 0.00e+00 NNZ =      163 SPT = 0.9735 Tmv

## The ``SGF`` Mode

Here we use ``get_ghf_integrals`` function to get the integrals.

In [7]:
from pyscf import gto, scf

mol = gto.M(atom="N 0 0 0; N 0 0 1.1", basis="sto3g", symmetry="d2h", verbose=0)
mf = scf.GHF(mol).run(conv_tol=1E-14)
ncas, n_elec, spin, ecore, h1e, g2e, orb_sym = itg.get_ghf_integrals(mf,
    ncore=0, ncas=None, g2e_symm=8)

driver = DMRGDriver(scratch="./tmp", symm_type=SymmetryTypes.SGF, n_threads=4)
driver.initialize_system(n_sites=ncas, n_elec=n_elec, spin=spin, orb_sym=orb_sym)

mpo = driver.get_qc_mpo(h1e=h1e, g2e=g2e, ecore=ecore, iprint=1)
energies = run_dmrg(driver, mpo)
print('DMRG energy = %20.15f' % energies)

integral symmetrize error =  2.1868782025982333e-13
integral cutoff error =  0.0
mpo terms =       5558

Build MPO | Nsites =    20 | Nterms =       5558 | Algorithm = FastBIP | Cutoff = 1.00e-20
 Site =     0 /    20 .. Mmpo =     7 DW = 0.00e+00 NNZ =        7 SPT = 0.0000 Tmvc = 0.001 T = 0.006
 Site =     1 /    20 .. Mmpo =    20 DW = 0.00e+00 NNZ =       19 SPT = 0.8643 Tmvc = 0.001 T = 0.005
 Site =     2 /    20 .. Mmpo =    47 DW = 0.00e+00 NNZ =       49 SPT = 0.9479 Tmvc = 0.001 T = 0.006
 Site =     3 /    20 .. Mmpo =    62 DW = 0.00e+00 NNZ =      251 SPT = 0.9139 Tmvc = 0.001 T = 0.008
 Site =     4 /    20 .. Mmpo =    81 DW = 0.00e+00 NNZ =      273 SPT = 0.9456 Tmvc = 0.001 T = 0.008
 Site =     5 /    20 .. Mmpo =   104 DW = 0.00e+00 NNZ =      357 SPT = 0.9576 Tmvc = 0.001 T = 0.008
 Site =     6 /    20 .. Mmpo =   129 DW = 0.00e+00 NNZ =      561 SPT = 0.9582 Tmvc = 0.001 T = 0.009
 Site =     7 /    20 .. Mmpo =   126 DW = 0.00e+00 NNZ =     2148 SPT = 0.8678 Tmv

## Relativistic DMRG

For relativistic DMRG, we use ``get_dhf_integrals`` function to get the integrals. We use the ``SGFCPX`` Mode in ``block2`` to execute DMRG. Note that the integrals, MPO, and MPS will all contain complex numbers in this mode.

In [8]:
from pyscf import gto, scf

mol = gto.M(atom="N 0 0 0; N 0 0 1.1", basis="sto3g", symmetry="d2h", verbose=0)
mf = scf.DHF(mol).set(with_gaunt=True, with_breit=True).run(conv_tol=1E-12)
ncas, n_elec, spin, ecore, h1e, g2e, orb_sym = itg.get_dhf_integrals(mf,
    ncore=0, ncas=None, pg_symm=False)

driver = DMRGDriver(scratch="./tmp", symm_type=SymmetryTypes.SGFCPX, n_threads=4)
driver.initialize_system(n_sites=ncas, n_elec=n_elec, spin=spin, orb_sym=orb_sym)

mpo = driver.get_qc_mpo(h1e=h1e, g2e=g2e, ecore=ecore, iprint=1)
energies = run_dmrg(driver, mpo)
print('DMRG energy = %20.15f' % energies)

integral symmetrize error =  0.0
integral cutoff error =  0.0
mpo terms =      25159

Build MPO | Nsites =    20 | Nterms =      25159 | Algorithm = FastBIP | Cutoff = 1.00e-20
 Site =     0 /    20 .. Mmpo =     9 DW = 0.00e+00 NNZ =        9 SPT = 0.0000 Tmvc = 0.001 T = 0.010
 Site =     1 /    20 .. Mmpo =    28 DW = 0.00e+00 NNZ =       23 SPT = 0.9087 Tmvc = 0.002 T = 0.011
 Site =     2 /    20 .. Mmpo =    63 DW = 0.00e+00 NNZ =      322 SPT = 0.8175 Tmvc = 0.002 T = 0.023
 Site =     3 /    20 .. Mmpo =    78 DW = 0.00e+00 NNZ =      437 SPT = 0.9111 Tmvc = 0.003 T = 0.026
 Site =     4 /    20 .. Mmpo =    97 DW = 0.00e+00 NNZ =      529 SPT = 0.9301 Tmvc = 0.003 T = 0.019
 Site =     5 /    20 .. Mmpo =   120 DW = 0.00e+00 NNZ =      734 SPT = 0.9369 Tmvc = 0.003 T = 0.019
 Site =     6 /    20 .. Mmpo =   147 DW = 0.00e+00 NNZ =      849 SPT = 0.9519 Tmvc = 0.003 T = 0.018
 Site =     7 /    20 .. Mmpo =   178 DW = 0.00e+00 NNZ =     1071 SPT = 0.9591 Tmvc = 0.005 T = 0.024

## Expectation and N-Particle Density Matrix

Once the optimized MPS is obtained, we can compute the expectation value on it, including its norm, the energy expectation, $\langle S^2 \rangle$, N-particle density matrix, or any operator that can be constructed as an MPO.

In this example, we compute the triplet state.

In [9]:
from pyscf import gto, scf

mol = gto.M(atom="N 0 0 0; N 0 0 1.1", basis="sto3g", symmetry="d2h", verbose=0)
mf = scf.RHF(mol).run(conv_tol=1E-14)
ncas, n_elec, spin, ecore, h1e, g2e, orb_sym = itg.get_rhf_integrals(mf,
    ncore=0, ncas=None, g2e_symm=8)

spin = 2

driver = DMRGDriver(scratch="./tmp", symm_type=SymmetryTypes.SU2, n_threads=4)
driver.initialize_system(n_sites=ncas, n_elec=n_elec, spin=spin, orb_sym=orb_sym)

mpo = driver.get_qc_mpo(h1e=h1e, g2e=g2e, ecore=ecore, iprint=0)

ket = driver.get_random_mps(tag="GS", bond_dim=250, nroots=1)
bond_dims = [250] * 4 + [500] * 4
noises = [1e-4] * 4 + [1e-5] * 4 + [0]
thrds = [1e-10] * 8
energies = driver.dmrg(
    mpo,
    ket,
    n_sweeps=20,
    bond_dims=bond_dims,
    noises=noises,
    thrds=thrds,
    iprint=0,
)

print('DMRG energy = %20.15f' % energies)

impo = driver.get_identity_mpo()

norm = driver.expectation(ket, impo, ket)
ener = driver.expectation(ket, mpo, ket)

print('Norm = %20.15f' % norm)
print('Energy expectation = %20.15f' % (ener / norm))

# <S^2> [ in spin-adapted mode this is always S(S+1) ]
ssq_mpo = driver.get_spin_square_mpo(iprint=0)
ssq = driver.expectation(ket, ssq_mpo, ket)
print('<S^2> expectation = %20.15f' % (ssq / norm))

DMRG energy = -106.939132859666657
Norm =    1.000000000000000
Energy expectation = -106.939132859666657
<S^2> expectation =    2.000000000000000


We can also evaluate expectation of arbitray operator such as the occupancy in the first orbital

$$
\hat{N}_0 = a^\dagger_{0\alpha} a_{0\alpha} + a^\dagger_{0\beta} a_{0\beta}
= \sqrt{2} \big(a_0^\dagger\big)^{[1/2]} \otimes_{[0]} \big(a_0\big)^{[1/2]}
$$

In [10]:
b = driver.expr_builder()
b.add_term("(C+D)0", [0, 0], np.sqrt(2))
n_mpo = driver.get_mpo(b.finalize(), iprint=0)

n_0 = driver.expectation(ket, n_mpo, ket)
print('N0 expectation = %20.15f' % (n_0 / norm))

N0 expectation =    1.999995824363186


We can then verify this number using 1PDM:

In [11]:
pdm1 = driver.get_1pdm(ket)
print('N0 expectation from 1pdm = %20.15f' % pdm1[0, 0])

N0 expectation from 1pdm =    1.999995824363186


We can compute the 3PDM and compare the result with the FCI 3PDM. Note that in ``pyscf`` the 3PDM is defined as 

$$
\mathrm{DM}_{ijklmn} := \langle E_{ij} E_{kl} E_{mn} \rangle
$$

So we have to use the same convention in ``block2`` by setting the ``su2_coupling`` parameter in ``block2`` to ``((C+D)0+%s)0``, which will set the spin coupling of 3PDM to ``((C+D)0+((C+D)0+(C+D)0)0)0``.


In [12]:
pdm3_b2 = driver.get_3pdm(ket, iprint=0, su2_coupling="((C+D)0+%s)0")

from pyscf import fci

mx = fci.addons.fix_spin_(fci.FCI(mf), ss=2)
mx.kernel(h1e, g2e, ncas, nelec=n_elec, nroots=3, tol=1E-12)
print(mx.e_tot)
pdm3_fci = fci.rdm.make_dm123('FCI3pdm_kern_sf', mx.ci[0], mx.ci[0], ncas, n_elec)[2]

print('diff = ', np.linalg.norm(pdm3_fci - pdm3_b2))

[-106.93913286 -106.85412245 -106.70055113]
diff =  8.969447473020382e-06
